In [103]:
# import necessary libs
import torch
import pickle
import numpy as np
from dis import dis
from astyx_utils import *
import matplotlib.pyplot as plt
from PIL import Image

In [104]:
# Define constants
DATA_DIR = "./data/"
IMAGE_DIR = DATA_DIR + "image/"
PCD_DIR = DATA_DIR + "pcd/"
BOX2D_DIR = DATA_DIR + "box2d/"
BOX3D_DIR = DATA_DIR + "box3d/"

In [105]:
# Getting filename for image, pcd, box2d and box3d
file_list_image=os.listdir(IMAGE_DIR)
file_list_no_ex = [x.split('.')[0] for x in file_list_image]
file_list_pcd = [x+".npy" for x in file_list_no_ex]
file_list_box2d = [x+"_box2d.pickle" for x in file_list_no_ex]
file_list_box3d = [x+"_box3d.pickle" for x in file_list_no_ex]

In [106]:
# Read images from camera
image_list = []
for path in file_list_image:
    img = plt.imread(IMAGE_DIR + path)
    image_list.append(img)
images = np.array(image_list)

In [107]:
# Read pcd from lidar
pcd_list = []
for path in file_list_pcd:
    pcd = np.load(PCD_DIR + path)
    pcd_list.append(pcd)
pcds = np.array(pcd_list)

In [108]:
# Read 2d boxes from Detectron
box2d_list = []
for path in file_list_box2d:
    with open(BOX2D_DIR + path, 'rb') as handle:
        box2d = pickle.load(handle)
        box2d_list.append(box2d.numpy())
boxes2d = np.array(box2d_list)
boxes2d

array([[[ 601.7733 ,  499.3728 ,  672.74225,  535.5947 ],
        [ 249.68852,  494.4106 ,  298.2574 ,  526.0343 ],
        [ 904.0054 ,  472.43103, 1078.1804 ,  561.80304]]], dtype=float32)

In [109]:
# Read 3d boxes center from OpenPCDet with pointpillar
box3d_list = []
for path in file_list_box3d:
    with open(BOX3D_DIR + path, 'rb') as handle:
        box3d = pickle.load(handle)
        box3d_list.append(box3d.numpy()[:,:3])
boxes3d = np.array(box3d_list)

In [110]:
boxes3d_to_img = []
for frame in boxes3d:
    frame_list = lidar2CameraOurs(frame[:, [1, 2, 0]])
    boxes3d_to_img.append(frame_list)

In [111]:
def detect_inlier(boxes2d, boxes3d):
    """
    Detect from the 3D boudning boexes if they are inside the 2D bounding boxes
    Args:
    boxes2d : np.array of shape [n, 4]
              boxes2d element [x1, y1, x2, y2]
              x1 -------- y1
               |          |
               |          |
               |          |
              x2 -------- y2
    boxes3d : np.array of shape [m, 2]
              boxes3d element [x, y, z]
    Return:
    list : [boolean] indicating if 3d boxes are inliers
    """
    result = []
    for box3d in boxes3d:
        inlier = False
        for box2d in boxes2d:
            if box3d[0] >= box2d[0] and box3d[0] <= box2d[2] and\
               box3d[1] >= box2d[1] and box3d[1] <= box2d[3]:
               inlier = True
        result.append(inlier)
    return result

In [112]:
box2d

tensor([[ 601.7733,  499.3728,  672.7422,  535.5947],
        [ 249.6885,  494.4106,  298.2574,  526.0343],
        [ 904.0054,  472.4310, 1078.1804,  561.8030]])

In [115]:
sum(detect_inlier(boxes2d[0], boxes3d_to_img[0]))

3